In [17]:
import pandas as pd
import maidenhead as mh

# Define some constants
SILVER_IN_PATH = '/usr/datalake/silver/stormevents/csvfiles/severe_dillon.csv'
SILVER_OUT_PATH = '/usr/datalake/silver/stormevents/csvfiles/severe_maidenhead.csv'

In [18]:
df = pd.read_csv(SILVER_IN_PATH)
df.head()

,EFFECTIVE,BEGIN_LAT,BEGIN_LON,EF_0,EF_1,EF_2,EF_3,EF_4,EF_5,HAIL_MINOR,HAIL_SIGNIFICANT,WIND_MINOR,WIND_SIGNIFICANT
0,2000-08-03 14:10:00,32.4500,-89.6500,0,0,0,0,0,0,0,0,1,0
1,2000-08-09 14:05:00,32.7000,-88.6500,0,0,0,0,0,0,0,0,1,0
2,2000-08-09 16:15:00,31.8667,-88.7167,0,0,0,0,0,0,0,0,1,0
3,2000-02-13 23:10:00,35.7833,-83.5500,0,0,0,0,0,0,0,0,1,0
4,2000-02-13 23:15:00,35.1833,-83.9333,0,0,0,0,0,0,0,0,1,0


In [20]:
df['MAIDENHEAD'] = df.apply(lambda x: mh.to_maiden(x.BEGIN_LAT, x.BEGIN_LON, 2), axis=1)
df['EFFECTIVE'] = pd.to_datetime(df['EFFECTIVE'])
df['EFFECTIVE'] = df['EFFECTIVE'].dt.round('h')
df.head()

,EFFECTIVE,BEGIN_LAT,BEGIN_LON,EF_0,EF_1,EF_2,EF_3,EF_4,EF_5,HAIL_MINOR,HAIL_SIGNIFICANT,WIND_MINOR,WIND_SIGNIFICANT,MAIDENHEAD
0,2000-08-03 14:00:00,32.4500,-89.6500,0,0,0,0,0,0,0,0,1,0,EM52
1,2000-08-09 14:00:00,32.7000,-88.6500,0,0,0,0,0,0,0,0,1,0,EM52
2,2000-08-09 16:00:00,31.8667,-88.7167,0,0,0,0,0,0,0,0,1,0,EM51
3,2000-02-13 23:00:00,35.7833,-83.5500,0,0,0,0,0,0,0,0,1,0,EM85
4,2000-02-13 23:00:00,35.1833,-83.9333,0,0,0,0,0,0,0,0,1,0,EM85


In [21]:
df_maidenhead = df.drop(labels=['BEGIN_LAT', 'BEGIN_LON'], axis=1)
df_maidenhead.shape

(721953, 12)

In [22]:
df_maidenhead.head()

,EFFECTIVE,EF_0,EF_1,EF_2,EF_3,EF_4,EF_5,HAIL_MINOR,HAIL_SIGNIFICANT,WIND_MINOR,WIND_SIGNIFICANT,MAIDENHEAD
0,2000-08-03 14:00:00,0,0,0,0,0,0,0,0,1,0,EM52
1,2000-08-09 14:00:00,0,0,0,0,0,0,0,0,1,0,EM52
2,2000-08-09 16:00:00,0,0,0,0,0,0,0,0,1,0,EM51
3,2000-02-13 23:00:00,0,0,0,0,0,0,0,0,1,0,EM85
4,2000-02-13 23:00:00,0,0,0,0,0,0,0,0,1,0,EM85


In [23]:
df_grouped = df_maidenhead.groupby(['EFFECTIVE', 'MAIDENHEAD']).max()
df_grouped.shape

(300724, 10)

In [24]:
df_grouped.head()

EF_0  EF_1  EF_2  EF_3  EF_4  EF_5  \
EFFECTIVE           MAIDENHEAD                                       
2000-01-02 17:00:00 EM45           0     0     0     0     0     0   
2000-01-02 18:00:00 EM45           0     0     0     0     0     0   
2000-01-03 00:00:00 EM03           0     0     0     0     0     0   
                    EM35           0     0     0     0     0     0   
                    EM75           0     0     0     0     0     0   

                                HAIL_MINOR  HAIL_SIGNIFICANT  WIND_MINOR  \
EFFECTIVE           MAIDENHEAD                                             
2000-01-02 17:00:00 EM45                 0                 0           1   
2000-01-02 18:00:00 EM45                 0                 0           1   
2000-01-03 00:00:00 EM03                 1                 0           0   
                    EM35                 1                 0           0   
                    EM75                 0                 0           1   

                                WIND_SIGNIFICANT  
EFFECTIVE           MAIDENHEAD                    
2000-01-02 17:00:00 EM45                       0  
2000-01-02 18:00:00 EM45                       0  
2000-01-03 00:00:00 EM03                       0  
                    EM35                       0  
                    EM75                       0

In [25]:
df_grouped.to_csv(SILVER_OUT_PATH)